# Machine Learning: Training Models

This notebook summarizes key theoretical concepts and practical examples from Chapter 4 on training models, covering:
- Linear Regression and its training algorithms
- Gradient Descent variants
- Polynomial Regression
- Regularization: Ridge, Lasso, Elastic Net
- Logistic Regression
- Softmax Regression

Code examples use NumPy and Scikit-Learn to illustrate model training and evaluation.

## Overview
Understanding how machine learning models are trained helps in model selection, hyperparameter tuning, debugging, and improving performance. We will explore classical training methods and modern optimization techniques.

## 1. Linear Regression Model

The Linear Regression model predicts an output \$ y \$ as a weighted sum of input features plus a bias term:

$$ y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n $$

Where:
- \$ \hat{y} \$ is the predicted value
- \$ x_i \$ are feature values
- \$ \theta_j \$ are model parameters (weights and bias)

In vectorized form:

$$ \hat{y} = h_{\theta}(x) = \theta^T x $$

with \$ x_0 = 1 \$ representing the bias feature.

We train the model by minimizing the Mean Squared Error (MSE) between predictions and actual values over the training set.

In [1]:
# Example: Creating a simple linear dataset
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
m = 100  # number of instances
X = 2 * np.random.rand(m, 1)  # feature
y = 4 + 3 * X + np.random.randn(m, 1)  # target with noise

plt.scatter(X, y)
plt.xlabel('Feature X')
plt.ylabel('Target y')
plt.title('Random Linear Dataset')
plt.show()

## 2. Training Linear Regression

### 2.1 Normal Equation (Closed-Form Solution)
A direct mathematical solution for \$ \theta \$ that minimizes the MSE cost function is given by:

$$ \theta = (X^T X)^{-1} X^T y $$

where \$ X \$ is the matrix of all training features with a leading column of ones (for bias). This method is fast for small feature sets, but costly for many features (computational complexity \$ O(n^2.4) \text{ to } O(n^3) \$).

Example using NumPy:

In [2]:
X_b = np.c_[np.ones((m, 1)), X]  # add bias feature x0=1
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(f"Theta computed from Normal Equation:\n{theta_best}")

Theta computed from Normal Equation:
[[4.21509616]
 [2.77011339]
]


### 2.2 Using Scikit-Learn's LinearRegression
Scikit-Learn provides a simple interface to train Linear Regression models, which internally uses efficient numerical methods (SVD).

In [3]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
print(f"Intercept: {lin_reg.intercept_}")
print(f"Coefficients: {lin_reg.coef_}")

Intercept: [4.21509616]
Coefficients: [[2.77011339]]


## 3. Gradient Descent

Instead of solving a matrix equation, Gradient Descent (GD) optimizes the parameters iteratively by taking steps proportional to the negative gradient of the cost function.

- Start with random \$ \theta \$
- Compute gradient \$ \nabla_{\theta} \text{MSE} \$
- Update \$ \theta \leftarrow \theta - \eta \times \nabla_{\theta} \text{MSE} \$, where \$ \eta \$ is the learning rate
- Repeat until convergence

The MSE gradient for all parameters can be computed as:

$$ \nabla_{\theta} \text{MSE} = \frac{2}{m} X^T (X \theta - y) $$

In [4]:
# Batch Gradient Descent implementation
eta = 0.1  # learning rate
n_iterations = 1000

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

print(f"Theta found by Gradient Descent:\n{theta}")

Theta found by Gradient Descent:
[[4.21509616]
 [2.77011339]
]


### Choosing the Learning Rate
- Too small: slow convergence
- Too large: may diverge or oscillate

Proper learning rate is essential. Often chosen by trial or grid search.

## 4. Gradient Descent Variants

### a) Batch Gradient Descent
Computes gradients using the **full** training set each step. Accurate but slow on large datasets.

### b) Stochastic Gradient Descent (SGD)
Updates parameters using one random instance at a time. Much faster iteration, noisy convergence, good for large/out-of-memory datasets. May oscillate around minimum.

### c) Mini-batch Gradient Descent
Compromise: updates with small batches of instances. Benefits from hardware acceleration, smoother than SGD, often used in deep learning.

In [5]:
# Simple Stochastic Gradient Descent example
n_epochs = 50
t0, t1 = 5, 50  # learning schedule parameters

def learning_schedule(t):
    return t0 / (t + t1)

theta_sgd = np.random.randn(2, 1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta_sgd) - yi)
        eta = learning_schedule(epoch * m + i)
        theta_sgd = theta_sgd - eta * gradients

print(f"Theta found by Stochastic Gradient Descent:\n{theta_sgd}")

Theta found by Stochastic Gradient Descent:
[[4.21076011]
 [2.74856079]
]


## 5. Polynomial Regression

To fit nonlinear data while still using linear models, we can add polynomial features (powers and interaction terms) of the original features.

Example: Original feature \$ x \$, add \$ x^2, x^3, ... \$ to features.

This increases model complexity and risk of overfitting, which can be detected using learning curves.

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Generate a nonlinear dataset (quadratic)
m = 100
X_nl = 6 * np.random.rand(m, 1) - 3
y_nl = 0.5 * X_nl**2 + X_nl + 2 + np.random.randn(m, 1)

# Create polynomial features of degree 2
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X_nl)

# Fit Linear Regression on polynomial features
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_poly, y_nl)
print(f"Intercept: {lin_reg_poly.intercept_}")
print(f"Coefficients: {lin_reg_poly.coef_}")

Intercept: [1.78134581]
Coefficients: [[0.93366893 0.56456263]]


## 6. Learning Curves

Plots of training and validation errors as a function of training set size help diagnose:
- Underfitting: High error on train and validation with curves close together
- Overfitting: Low train error, high validation error, with a gap between curves

Example function to plot learning curves:

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    train_errors, val_errors = [], []
    
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    
    plt.plot(np.sqrt(train_errors), 'r-+', linewidth=2, label='Train')
    plt.plot(np.sqrt(val_errors), 'b-', linewidth=3, label='Validation')
    plt.legend()
    plt.xlabel('Training set size')
    plt.ylabel('RMSE')
    plt.title('Learning Curves')
    plt.show()

### Example of learning curves for Linear Regression on nonlinear data


In [8]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X_nl, y_nl)

## 7. Regularization

Regularization helps prevent overfitting by constraining model weights.

### a) Ridge Regression (L2 regularization)
- Adds \$ \alpha \times \frac{1}{2} \sum_{i=1}^n \theta_i^2 \$ penalty to cost function
- Shrinks weights towards zero but does not eliminate them
- Controls model complexity by \$ \alpha \$

### b) Lasso Regression (L1 regularization)
- Adds \$ \alpha \sum_{i=1}^n |\theta_i| \$ penalty
- Can set some weights exactly to zero (feature selection)

### c) Elastic Net
- Combines L1 and L2 penalties controlled by ratio \$ r \$
- Useful when features are correlated

Regularization improves generalization by reducing variance at the cost of some bias.

## 8. Early Stopping

Used to regularize iterative training by stopping training when validation error stops improving, avoiding overfitting. Can save the best model seen during training.

## 9. Logistic Regression

Used for binary classification, Logistic Regression estimates the probability \$ p \$ of belonging to positive class:

$$ p = h_{\theta}(x) = \sigma(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}} $$

Predictions:
- Predict 1 if \$ p \geq 0.5 \$
- Predict 0 otherwise

The cost function is log loss (cross-entropy):

$$ J(\theta) = - \frac{1}{m} \sum_{i=1}^m \left[ y_i \log p_i + (1-y_i) \log (1-p_i) \right] $$

No closed form; trained via Gradient Descent.

In [9]:
# Logistic Regression example with iris dataset
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

iris = datasets.load_iris()
X_log = iris['data'][:, 3:]  # petal width
y_log = (iris['target'] == 2).astype(int)  # Iris virginica?

log_reg = LogisticRegression()
log_reg.fit(X_log, y_log)

import matplotlib.pyplot as plt
import numpy as np

X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)

plt.plot(X_new, y_proba[:, 1], 'g-', label='Iris virginica')
plt.plot(X_new, y_proba[:, 0], 'b--', label='Not Iris virginica')
plt.legend()
plt.title('Logistic Regression Estimated Probabilities')
plt.xlabel('Petal width (cm)')
plt.ylabel('Probability')
plt.show()

## 10. Softmax Regression

Generalizes Logistic Regression to multi-class classification by computing a score for each class and applying the softmax function:

$$ p_k = \frac{e^{s_k(x)}}{\sum_{j=1}^K e^{s_j(x)}} $$

Where \$ s_k(x) = \theta_k^T x \$ is the score for class \$ k \$.

Predict the class with highest probability.

The training minimizes cross-entropy loss across classes.

In [10]:
X_multi = iris['data'][:, (2, 3)]  # petal length and width
y_multi = iris['target']

softmax_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=10)
softmax_reg.fit(X_multi, y_multi)

print(f"Prediction for a flower with petals 5cm x 2cm: {softmax_reg.predict([[5, 2]])}")
print(f"Class probabilities: {softmax_reg.predict_proba([[5, 2]])}")

Prediction for a flower with petals 5cm x 2cm: [2]
Class probabilities: [[6.38014896e-07 5.74929995e-02 9.42506362e-01]]


This concludes the summary and explanation of core supervised learning model training techniques from Chapter 4. The notebook serves as a hands-on companion to understand theory and implementation of these foundational ML concepts.